# Task_03. Collect SARS-CoV-2 N protein & (optional) RNA-dependent RNA proteins 

## Homework_03_Procedures
<ol>
     <li> Read GCF_009858895.2 genomic info.
     <li> Make N-protein cDNA sequence.
     <li> Translate it to protein sequence. 
     <li> Q1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna
     <li> Based on this info, extract N proteins cDNA/proteins from other SARS-CoV2 genomes (either 02_03 or 03_27).
     <li> Q2. Which pairs are most similar ones? Which pairs are most different ones?
     <li> Based on this info, extract N proteins cDNA/proteins from other SARS & MERS genomes.
     <li> Q3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
     <li> An optional assignment for RdRP for Q1, Q2 and Q3
     <li> Extraction results of N-protein cDNA and N-protein sequences.
              
</ol>




## Data files
<ul>
    <li> The genome of Covid-19: '../data/2019nCoV_genomes.2020_03_27.fa'   
    <li> The protein of Covid-19: '../data/2019nCoV_proteins.2020_03_27.fa'
    <li> The genome of MERS: '../data/MERS_genomes.2020_02_03.fa'
    <li> The genome of SARS: '../data/SARS_genomes.2020_02_03.fa'
    <li> Reference file for Covid-19: GCF_009858895 files
         
</ul>
    

In [42]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [43]:
# The function to write sequences to a FASTA file
def write_fasta(tmp_filename, genome_list):
    f = open(tmp_filename, 'w')
    for genome in list(genome_list.keys()):
        if len(genome_list[genome]) == 1:
            f.write(">{}\n".format(genome))
            f.write("{}\n".format(genome_list[genome][0]))
        else:
            for idx in range(len(genome_list[genome])):
                f.write(">{} #{}\n".format(genome, idx+1))
                f.write("{}\n".format(genome_list[genome][idx]))
    f.close()

In [44]:
# Read GCF_009858895.2 genomic info.
filename_reference = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
ref_genome = read_fasta(filename_reference)

In [45]:
# Find N-protein index (starts with 0) in GCF_009858895.2 from the gff file.
filename_ref_index = '../data/GCF_009858895.2_ASM985889v3_genomic.gff'
def get_index(temp_filename, protein_name):
    f = open(temp_filename, 'r')
    for line in f.readlines():
        if "gene" in line and "Name={}".format(protein_name) in line:
            return int(line.split()[3])-1, int(line.split()[4])-1
print(get_index(filename_ref_index, 'N'))

(28273, 29532)


In [46]:
# Get N-protein cDNA sequece of GCF_009858895.2 and extract it to a file
index = get_index(filename_ref_index, 'N')
for sequence in ref_genome.values():
    ref_N_sequence = sequence[index[0]:index[1]+1]
ref_cDNA = {'N_protein' : [ref_N_sequence]}
write_fasta("ref_cDNA.fa", ref_cDNA)

f = open("ref_cDNA.fa", 'r')
print(f.readline())
print(f.readline()[:25])

>N_protein

ATGTCTGATAATGGACCCCAAAATC


In [47]:
# Translate it to protein sequence
'''
I commanded in Ubuntu following lines to translate ref_cDNA.fa which contains the sequence of N-protein.

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
transeq --sequence ref_cDNA.fa --outseq ref_protein.fa

'''
print('')

In [48]:
#HW 3-1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna

filename_my_ref_protein = 'ref_protein.fa'
my_ref_protein = read_fasta(filename_my_ref_protein)
#print(my_ref_protein)

filename_ref_protein = '../data/GCF_009858895.2_ASM985889v3_protein.faa'
ref_protein = read_fasta(filename_ref_protein)

if my_ref_protein['N_protein_1'][:-1] == ref_protein['YP_009724397.2 nucleocapsid phosphoprotein [Wuhan seafood market pneumonia virus]']:
    print("Translated cDNA of N-protein I got and the reference N-protein are same")

Translated cDNA of N-protein I got and the reference N-protein are same


In [49]:
# The function to print result of blast -outfmt7, shows results with maximum and minimum accuracies.
def print_blastresult(temp_filename, compare_object):
    f = open(temp_filename, 'r')
    percentages = []
    for line in f.readlines():
        if line.startswith(compare_object):
            percentages.append(float(line.split()[2]))
    max_percentage = max(percentages)
    min_percentage = min(percentages)
    f.close()

    f = open(temp_filename, 'r')
    highests = []
    lowests = []
    for line in f.readlines():
        if line.startswith(compare_object):
            if float(line.split()[2]) == max_percentage:
                highests.append(line.split()[1])
            if float(line.split()[2]) == min_percentage:
                lowests.append(line.split()[1])
    print(highests, "are the most similar ones with {}% of identity\n".format(max_percentage))
    print(lowests, "are the most different ones with {}% of identity\n".format(min_percentage))
    f.close()

In [50]:
# The function to get a dictionary of sequences from blast -outfmt7,
# for extracting N proteins cDNA/proteins from sequences genomes or proteins.
# This function is not used for analyizing this time. (I only used the blastn and blastp for analyzing)
# Actual results using this fuchtion to get the sequences are attached in the very bottom.

def get_blasted(temp_filename, compare_object, genome_or_protein_list):
    f = open(temp_filename, 'r')
    sequences = {}
    for line in f.readlines():
        if line.startswith(compare_object):
            name = str(line.split()[1])
            index = (int(line.split()[8])-1, int(line.split()[9])-1)
            sequences[name] = genome_or_protein_list[name][index[0]:index[1]+1]
    return sequences

In [51]:
#Based on this info, extract N proteins cDNA/proteins from other SARS-CoV2 genomes (03_27).
# HW 3-2. Which pairs are most similar ones? Which pairs are most different ones?
# (1/2) To compare N proteins cDNA

'''
I commanded in Ubuntu following line to compare N-protein cDNA sequence of the reference file and 03_27 file.

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
blastn -db ../data/2019nCoV_genomes.2020_03_27.blastdb -query ref_cDNA.fa -outfmt 7 -out cDNA_N_protein_03_27_nblast.nbl
'''

filename_N_0327_nblast = 'cDNA_N_protein_03_27_nblast.nbl'
print_blastresult(filename_N_0327_nblast, 'N_protein')

genome_list = read_fasta('../data/2019nCoV_genomes.2020_03_27.fa')
genome_list_0327 = {name[:8] : genome_list[name] for name in genome_list.keys()}
#print(get_blasted(filename_N_0327_nblast, 'N_protein', genome_list_0327))

['MT049951', 'MT066176', 'MT066175', 'MT072688', 'MT093571', 'MT093631', 'MT123292', 'LC528232', 'LC528233', 'MT126808', 'MT135044', 'MT135042', 'MT135041', 'MT152824', 'MT050493', 'MT012098', 'MT066156', 'MT121215', 'MT163718', 'MT163717', 'MT163716', 'MT163719', 'MT192773', 'MT192759', 'MT192765', 'MT192772', 'MT240479', 'MT246490', 'MT246489', 'MT246488', 'MT246487', 'MT246486', 'MT246484', 'MT246482', 'MT246481', 'MT246480', 'MT246479', 'MT246478', 'MT246477', 'MT246476', 'MT246475', 'MT246474', 'MT246473', 'MT246472', 'MT246471', 'MT246469', 'MT246468', 'MT246467', 'MT246466', 'MT246464', 'MT246462', 'MT246461', 'MT246460', 'MT246459', 'MT246458', 'MT246457', 'MT246455', 'MT246453', 'MT246452', 'MT246450', 'MT246449'] are the most similar ones with 100.0% of identity

['MT233522', 'MT246470', 'MT246451'] are the most different ones with 99.762% of identity



In [52]:
#Based on this info, extract N proteins cDNA/proteins from other SARS-CoV2 genomes (03_27).
# HW 3-2. Which pairs are most similar ones? Which pairs are most different ones?
# (2/2) To compare N proteins

'''
I commanded in Ubuntu following lines to compare N-protein sequence of the reference file and 03_27 file.

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
transeq --sequence ../data/2019nCoV_genomes.2020_03_27.fa --outseq 2019nCoV_genomes.2020_03_27_trans6.fa -frame 6

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
makeblastdb -dbtype prot -in 2019nCoV_genomes.2020_03_27_trans6.fa -out 2019nCoV_genomes.2020_03_27_trans6

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
blastp -db 2019nCoV_genomes.2020_03_27_trans6 -query ref_protein.fa -outfmt 7 -out N_protein_03_27_tblast.tbl

'''
filename_N_0327_tblast = 'N_protein_03_27_tblast.tbl'
print_blastresult(filename_N_0327_tblast, 'N_protein')

protein_list = read_fasta('2019nCoV_genomes.2020_03_27_trans6.fa')
protein_list_0327 = {name[:10] : protein_list[name] for name in protein_list.keys()}
#print(get_blasted(filename_N_0327_tblast, 'N_protein', protein_list_0327))

['MT049951_2', 'MT066176_2', 'MT066175_2', 'MT072688_2', 'MT093571_2', 'MT093631_1', 'MT123293_2', 'MT123291_2', 'MT123292_2', 'LC528232_2', 'LC528233_1', 'MT126808_2', 'MT135044_2', 'MT135042_2', 'MT135041_2', 'MT152824_3', 'MT050493_3', 'MT012098_1', 'MT066156_2', 'MT121215_2', 'MT163718_2', 'MT163717_2', 'MT163716_2', 'MT163719_2', 'MT192773_1', 'MT192759_2', 'MT192765_1', 'MT192772_2', 'MT240479_2', 'MT246490_3', 'MT246489_3', 'MT246488_2', 'MT246487_1', 'MT246486_3', 'MT246484_3', 'MT246482_2', 'MT246481_3', 'MT246480_1', 'MT246479_3', 'MT246478_1', 'MT246477_1', 'MT246476_2', 'MT246475_3', 'MT246474_1', 'MT246473_3', 'MT246472_1', 'MT246471_3', 'MT246469_1', 'MT246468_2', 'MT246467_2', 'MT246466_2', 'MT246464_2', 'MT246462_2', 'MT246461_3', 'MT246460_2', 'MT246459_2', 'MT246458_2', 'MT246457_3', 'MT246455_2', 'MT246454_2', 'MT246453_3', 'MT246452_1', 'MT246450_1', 'MT246449_3'] are the most similar ones with 100.0% of identity

['MT233522_2', 'MT246470_2', 'MT246451_2'] are the m

In [53]:
#Based on this info, extract N proteins cDNA/proteins from other SARS & MERS genomes..
# HW 3-3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
# (1/2) To compare SARS N proteins

'''
I commanded in Ubuntu following lines to compare N-protein sequence of the reference file and SARS file.

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
transeq --sequence ../data/SARS_genomes.2020_02_03.fa --outseq SARS_genomes.2020_02_03_trans6.fa -frame 6

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
makeblastdb -dbtype prot -in SARS_genomes.2020_02_03_trans6.fa -out SARS_genomes.2020_02_03_trans6.fa

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
blastp -db SARS_genomes.2020_02_03_trans6.fa -query ref_protein.fa -outfmt 7 -out N_protein_SARS_tblast.tbl

'''

filename_N_SARS_tblast = 'N_protein_SARS_tblast.tbl'
print_blastresult(filename_N_SARS_tblast, 'N_protein')

protein_list = read_fasta('SARS_genomes.2020_02_03_trans6.fa')
protein_list_SARS = {name[:12] : protein_list[name] for name in protein_list.keys()}
#print(get_blasted(filename_N_SARS_tblast, 'N_protein', protein_list_SARS))

['KU973692.1_3', 'DQ898174.1_1', 'FJ882949.1_1', 'FJ882934.1_3', 'FJ882937.1_2', 'FJ882935.1_1', 'FJ882933.1_2', 'FJ882947.1_3', 'FJ882946.1_1', 'FJ882927.1_2', 'FJ882932.1_3', 'FJ882939.1_3', 'FJ882936.1_3', 'FJ882938.1_2', 'FJ882941.1_1', 'FJ882929.1_1', 'FJ882944.1_3', 'FJ882940.1_3', 'FJ882954.1_2', 'FJ882960.1_3', 'FJ882955.1_3', 'FJ882931.1_3', 'FJ882928.1_2', 'FJ882926.1_2', 'FJ882930.1_2', 'AY362698.1_1', 'AY394850.2_2'] are the most similar ones with 90.544% of identity

['DQ084200.1_3', 'DQ022305.2_2'] are the most different ones with 89.573% of identity



In [55]:
#Based on this info, extract N proteins cDNA/proteins from other SARS & MERS genomes..
# HW 3-3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
# (2/2) To compare MERS N proteins

'''
I commanded in Ubuntu following lines to compare N-protein sequence of the reference file and MERS file.

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
transeq --sequence ../data/MERS_genomes.2020_02_03.fa --outseq MERS_genomes.2020_02_03_trans6.fa -frame 6

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
makeblastdb -dbtype prot -in MERS_genomes.2020_02_03_trans6.fa -out MERS_genomes.2020_02_03_trans6.fa

(base) ehojune@DESKTOP-71IDJ52:~/git/BME203-2020/03_Annotation$ 
blastp -db MERS_genomes.2020_02_03_trans6.fa -query ref_protein.fa -outfmt 7 -out N_protein_MERS_tblast.tbl

'''

filename_N_MERS_tblast = 'N_protein_MERS_tblast.tbl'
print_blastresult(filename_N_MERS_tblast, 'N_protein')

protein_list = read_fasta('MERS_genomes.2020_02_03_trans6.fa')
protein_list_MERS = {name[:12] : protein_list[name] for name in protein_list.keys()}
#print(get_blasted(filename_N_MERS_tblast, 'N_protein', protein_list_MERS))

['KX574227.1_3'] are the most similar ones with 50.142% of identity

['KX108941.1_2'] are the most different ones with 43.333% of identity



In [56]:
# Optional Assignment: Analysis for RdRP
# I got The sequence of RdRP from NCBI: https://www.ncbi.nlm.nih.gov/protein/YP_009725307.1
# All the commands in Ubuntu is ommitted.

RdRP_lower = """sadaqsflnr vcgvsaarlt pcgtgtstdv vyrafdiynd kvagfakflk tnccrfqekd
 eddnlidsyf vvkrhtfsny qheetiynll kdcpavakhd ffkfridgdm vphisrqrlt
 kytmadlvya lrhfdegncd tlkeilvtyn ccdddyfnkk dwydfvenpd ilrvyanlge
 rvrqallktv qfcdamrnag ivgvltldnq dlngnwydfg dfiqttpgsg vpvvdsyysl
 lmpiltltra ltaeshvdtd ltkpyikwdl lkydfteerl klfdryfkyw dqtyhpncvn
 clddrcilhc anfnvlfstv fpptsfgplv rkifvdgvpf vvstgyhfre lgvvhnqdvn
 lhssrlsfke llvyaadpam haasgnllld krttcfsvaa ltnnvafqtv kpgnfnkdfy
 dfavskgffk egssvelkhf ffaqdgnaai sdydyyrynl ptmcdirqll fvvevvdkyf
 dcydggcina nqvivnnldk sagfpfnkwg karlyydsms yedqdalfay tkrnviptit
 qmnlkyaisa knrartvagv sicstmtnrq fhqkllksia atrgatvvig tskfyggwhn
 mlktvysdve nphlmgwdyp kcdrampnml rimaslvlar khttccslsh rfyrlaneca
 qvlsemvmcg gslyvkpggt ssgdattaya nsvfnicqav tanvnallst dgnkiadkyv
 rnlqhrlyec lyrnrdvdtd fvnefyaylr khfsmmilsd davvcfnsty asqglvasik
 nfksvlyyqn nvfmseakcw tetdltkgph efcsqhtmlv kqgddyvylp ypdpsrilga
 gcfvddivkt dgtlmierfv slaidayplt khpnqeyadv fhlylqyirk lhdeltghml
 dmysvmltnd ntsrywepef yeamytphtv lq"""
RdRP_sequence = ''.join(RdRP_lower.split()).upper()
RdRP = {'RdRP' : [RdRP_sequence]}
write_fasta('ref_RdRP.fa', RdRP)

reference_proteins = read_fasta('../data/GCF_009858895.2_ASM985889v3_protein.faa')
for sequence in reference_proteins.values():
    if RdRP_sequence in sequence:
        print("RdRP I got and RdRP from the reference are same.\n")

print("**********tblastn result of RdRP with 2019nCoV-0327 genomes****************")
filename_RdRP_0327_tblastn = 'cDNA_RdRP_03_27_tblastn.nbl'
print_blastresult(filename_RdRP_0327_tblastn, 'RdRP')

print("**********blastp result of RdRP with 2019nCoV-0327 6frame proteins*********")
filename_RdRP_0327_blastp = 'RdRP_03_27_blastp.tbl'
print_blastresult(filename_RdRP_0327_blastp, 'RdRP')

print("**********blastp result of RdRP with SARS 6frame proteins******************")
filename_RdRP_SARS_blastp = 'RdRP_SARS_blastp.tbl'
print_blastresult(filename_RdRP_SARS_blastp, 'RdRP')

print("**********blastp result of RdRP with MERS 6frame proteins******************")
filename_RdRP_MERS_blastp = 'RdRP_MERS_blastp.tbl'
print_blastresult(filename_RdRP_MERS_blastp, 'RdRP')

RdRP I got and RdRP from the reference are same.

**********tblastn result of RdRP with 2019nCoV-0327 genomes****************
['MT049951', 'MT066176', 'MT066175', 'MT072688', 'MT093571', 'MT093631', 'MT123293', 'MT123291', 'MT123290', 'MT123292', 'LC528232', 'LC528233', 'MT126808', 'MT135044', 'MT135043', 'MT135042', 'MT135041', 'MT152824', 'MT050493', 'MT066156', 'MT121215', 'MT163718', 'MT163717', 'MT163716', 'MT163719', 'MT192773', 'MT192759', 'MT192772', 'MT233523', 'MT233519', 'MT240479', 'MT246489', 'MT246488', 'MT246486', 'MT246485', 'MT246482', 'MT246479', 'MT246478', 'MT246477', 'MT246476', 'MT246475', 'MT246474', 'MT246473', 'MT246472', 'MT246471', 'MT246469', 'MT246466', 'MT246464', 'MT246462', 'MT246461', 'MT246459', 'MT246458', 'MT246457', 'MT246456', 'MT246455', 'MT246454', 'MT246452'] are the most similar ones with 99.784% of identity

['MT246451'] are the most different ones with 99.568% of identity

**********blastp result of RdRP with 2019nCoV-0327 6frame proteins****

In [57]:
#This is the end of solving Q1, Q2, Q3 and the optional quetion.
#Really appriciate to you for reading all.
print("\n" * 10)

In [60]:
# Below is just to show the result of extractions of sequences of cDNA or N protein.
# This statements exists to satisfy the direction: 'extract N proteins cDNA/proteins from ...', in the slide.
# Nothing follows after this.

print("Extraction of cDNA sequences of N-protein from SARS-2019CoV2")
print(get_blasted(filename_N_0327_nblast, 'N_protein', genome_list_0327))
print("\n" * 30)

print("Extraction of N-protein sequences from SARS-2019CoV2")
print(get_blasted(filename_N_0327_tblast, 'N_protein', protein_list_0327))
print("\n" * 30)

print("Extraction of N-protein sequences from SARS")
print(get_blasted(filename_N_SARS_tblast, 'N_protein', protein_list_SARS))
print("\n" * 30)

print("Extraction of N-protein sequences from MERS")
print(get_blasted(filename_N_MERS_tblast, 'N_protein', protein_list_MERS))
print("This is the real end")

Extraction of cDNA sequences of N-protein from SARS-2019CoV2
{'MT049951': 'ATGTCTGATAATGGACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGAATGGAGAACGCAGTGGGGCGCGATCAAAACAACGTCGGCCCCAAGGTTTACCCAATAATACTGCGTCTTGGTTCACCGCTCTCACTCAACATGGCAAGGAAGACCTTAAATTCCCTCGAGGACAAGGCGTTCCAATTAACACCAATAGCAGTCCAGATGACCAAATTGGCTACTACCGAAGAGCTACCAGACGAATTCGTGGTGGTGACGGTAAAATGAAAGATCTCAGTCCAAGATGGTATTTCTACTACCTAGGAACTGGGCCAGAAGCTGGACTTCCCTATGGTGCTAACAAAGACGGCATCATATGGGTTGCAACTGAGGGAGCCTTGAATACACCAAAAGATCACATTGGCACCCGCAATCCTGCTAACAATGCTGCAATCGTGCTACAACTTCCTCAAGGAACAACATTGCCAAAAGGCTTCTACGCAGAAGGGAGCAGAGGCGGCAGTCAAGCCTCTTCTCGTTCCTCATCACGTAGTCGCAACAGTTCAAGAAATTCAACTCCAGGCAGCAGTAGGGGAACTTCTCCTGCTAGAATGGCTGGCAATGGCGGTGATGCTGCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAGGCCAAACTGTCACTAAGAAATCTGCTGCTGAGGCTTCTAAGAAGCCTCGGCAAAAACGTACTGCCACTAAAGCATACAATGTAACACAAGCTTTCGGCAGACGTGGTCCAGAACAAACCCAAGGAAATTTTGGGGACCAGGAACTAATCAGACAAGGAACTGATTACAAACATTGGCCGCAAATTGCACAATTTGCCC